# SPARQL generation via implicit schema representation and Generative AI
---
This notebook demonstrates one way to generate SPARQL queries from natural language questions. Here we focus on
prompting the model by showing the model examples of SPARQL queries (and their associated natural language questions)
and relying on the LLM to infer the graph schema.

If you are running this notebook outside of an AWS environment (e.g., on your laptop) then you will need
to set up your AWS credentials in ~/.aws/credentials.

If you are running this notebook inside of an AWS environment (e.g., inside Sagemaker Studio or as a Neptune notebook) then
use the "conda_pytorch_p310" kernel.

To access the Bedrock API from this notebook the notebook's execution role must have a policy to allow Bedrock access.
To find this notebook's execution role run the following code in this notebook:
```
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
```
and then go to the IAM console and add the policy `AmazonBedrockFullAccess`

In [1]:
# The first time you run this notebook you'll need to uncomment these lines
# to install the required Python dependencies:
# %pip install -q boto3==1.34.*
# %pip install -q botocore==1.34.*
# %pip install -q jupyter==1.0.*
# %pip install -q sagemaker==2.212.*
# %pip install -q jinja2==3.1.*
# %pip install -q ipykernel==6.29.*

In [51]:
from pathlib import Path
import importlib
from functools import partial

import utilities as u
# To reload utilities.py without restarting the notebook kernel use the following:
# importlib.reload(u)

import boto3
import sagemaker
import jinja2

## Set Neptune and Bedrock clients

### Get connection to Neptune database
And define a function to run a SPARQL query on that database

### Setup Bedrock client
And specify which model to use

In [54]:
sess = sagemaker.Session()
region = sess.boto_region_name
sm_client = boto3.client("sagemaker", region_name=region)
bedrock_runtime = boto3.client("bedrock-runtime", region_name=region)
# bedrock = boto3.client("bedrock", region_name=region)
jenv = jinja2.Environment(trim_blocks=True, lstrip_blocks=True)

model_id = "anthropic.claude-v2:1"
# model_id = "anthropic.claude-3-haiku-20240307-v1:0"
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
temperature = 0.1

### Define functions to generate SPARQL using LLM via Bedrock and run it against Neptune

In [64]:
prompt_template = (Path.cwd() / "resources" / "prompt.txt").read_text()


def generate_sparql_query(question: str) -> str:
    """
    Given a natural language question, use the LLM to transform that
    into a SPARQL query (using the prompt template) and return the query.
    """
    prompt = jenv.from_string(prompt_template).render(question=question)
    # print(f"prompt: <<{prompt}>>")
    response = u.run_bedrock(prompt, temperature, bedrock_runtime, model_id).strip()
    # print(f"response: <<<{response}>>>")
    try:
        idx = response.index("<sparql>")
        response = response[idx+8:]
        idx = response.index("</sparql>")
        response = response[:idx]
    except ValueError:
        pass
    return response


generate_and_run = partial(u.generate_and_run,
                           sparql_generator=generate_sparql_query,
                           sagemaker_session=sess)

## Try some queries

Here are some example queries that you can try. These queries are the same as those used for the few-shot prompting so they
are not a good guide to the model's ability to generalize beyond those examples. However, you can use these as a starting
point to explore the ability of the model to generate de novo SPARQL queries.

- What GO terms are associated with human proteins?
- What GO terms are associated with human proteins? Show me their names also.
- How many citations are there for papers by A. Bairoch?
- Show me all citations by A. Bairoch
- Show me all proteins that are located in the mitochondrian
- I'd like to see the entries for all proteins encoded by the gene FNDC3A
- Select all taxa from the UniProt taxonomy
- Select all taxa from the UniProt taxonomy; show me at most 7
- Show me at most 5 taxa from the UniProt taxonomy
- Select all bacterial taxa and their scientific names from the UniProt taxonomy
- Show me up to 10 human taxa and their scientific names from the UniProt taxonomy
- Select up to 10 bacterial taxa and their scientific names from the UniProt taxonomy
- Tell me all the different categories of databases
- Tell me all the different databases you know about
- Select all UniProt entries, and their organism and amino acid sequences (including isoforms), for _E. coli K12_ and all its strains
- Select the UniProt entry with the mnemonic 'A4_HUMAN'
- Select a mapping of UniProt to PDB entries using the UniProt cross-references to the PDB database
- Select all cross-references to external databases of the category '3D structure databases' of UniProt entries that are classified with the keyword 'Acetoin biosynthesis (KW-0005)'
- Select reviewed UniProt entries (Swiss-Prot), and their recommended protein name, that have a preferred gene name that contains the text 'DNA'
- Select reviewed UniProt entries (Swiss-Prot), and their recommended protein name, that have a preferred gene name that contains the word DNA. Show me the gene name too
- Show me the preferred gene name and disease annotation of all human UniProt entries that are known to be involved in a disease
- Select all human UniProt entries with a sequence variant that leads to a 'loss of function'
- Select all distinct human UniProt entries with a sequence variant that leads to a 'loss of function', show me the text of the annotation also
- Show me all human UniProt entries with a sequence variant that leads to a tyrosine to phenylalanine substitution
- Show me all human UniProt entries with a sequence variant that leads to a Tyr to phenylalanine substitution
- Select all UniProt entries with annotated transmembrane regions and the regions' begin and end coordinates on the canonical sequence
- Select all UniProt entries that were integrated on the 30th of November 2010
- Select all UniProt entries that were integrated on or before the 30th of November 2010
- Select all UniProt entries that were integrated on the month of November 2010
- Show me all UniProt entries that were added to the database on the month of November 2010
- Select the average number of cross-references to the PDB database of UniProt entries that have at least one cross-reference to the PDB database
- Select the number of UniProt entries for each of the EC (Enzyme Commission) second level categories
- Find all Natural Variant Annotations if associated via an evidence tag to an article with a pubmed identifier.
- Find where disease related proteins are known to be located in the cell
- How many reviewed entries (Swiss-Prot) are related to kinase activity?
- list all the Homo Sapiens proteins classified with "cholesterol biosynthetic process"
- list all the Homo Sapiens proteins classified with "cholesterol biosynthetic process". Include their names.
- list all the Homo Sapiens proteins classified with "cholesterol biosynthetic process". Don't include their names.
- find all the Homo Sapiens related proteins that have a Gene Ontology (GO) code
- look within the taxonomy tree, to see if there are any subclass records under Homo Sapiens. Return the scientific name also

In [65]:
generate_and_run("Show me all proteins that are located in the mitochondrian")

SPARQL query:

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX vg: <http://biohackathon.org/resource/vg#>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX uniprotkb: <http://purl.uniprot.org/uniprot/>
PREFIX uberon: <http://purl.obolibrary.org/obo/uo#>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX sp: <http://spinrdf.org/sp#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sio: <http://semanticscience.org/resource/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX schema: <http://schema.org/>
PREFIX sachem: <http://bioinfo.uochb.cas.cz/rdf/v1.0/sachem#>
PREFIX rh: <http://rdf.rhea-db.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pubmed: <http://rdf.ncbi.nlm.nih.gov/pubmed/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX pq: <htt

[]

In [66]:
generate_and_run("Select the average number of cross-references to the PDB "
                 "database of UniProt entries that have at least one cross-reference "
                 "to the PDB database")

SPARQL query:

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX vg: <http://biohackathon.org/resource/vg#>
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX uniprotkb: <http://purl.uniprot.org/uniprot/>
PREFIX uberon: <http://purl.obolibrary.org/obo/uo#>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX sp: <http://spinrdf.org/sp#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sio: <http://semanticscience.org/resource/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX schema: <http://schema.org/>
PREFIX sachem: <http://bioinfo.uochb.cas.cz/rdf/v1.0/sachem#>
PREFIX rh: <http://rdf.rhea-db.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX pubmed: <http://rdf.ncbi.nlm.nih.gov/pubmed/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX pq: <htt

[{'avgLinksToPdbPerEntry': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
   'type': 'literal',
   'value': '7.25071137429823886795'}}]